# 4-points metrics

### Functions

In [3]:
## "Import" the main structures needed
%run ./final_mainstructures.ipynb

"""Specific functions for the computations for 4-points metrics"""

class MetricOn4Points:
    """A metric on a 4-points set, defined with a system of splits constructed from the 
    predefined 6 splits that make the split-decomposition of 4 points metrics unique. 
    These splits are weighted according to the list of 6 weights provided in constructor."""
    def __init__(self, weights):
        self.X = FiniteSet(Set(['x1', 'x2', 'x3', 'x4']))
        self.S = SplitSystem(self.X)
        self.S.splits.extend([
        Split(self.X, Set(['x1']), weights[0]),
        Split(self.X, Set(['x2']), weights[1]),
        Split(self.X, Set(['x3']), weights[2]),
        Split(self.X, Set(['x4']), weights[3]),
        Split(self.X, Set(['x1', 'x2']), weights[4]),
        Split(self.X, Set(['x1', 'x3']), weights[5]) ])
        # The last possible one, Split(X, Set(['1', '4'])) is excluded in order to have a unique decomposition, 
        # see Bandelt&Dress p49
        self.d = self.S.d_alpha
        (self.d12, self.d13, self.d14, self.d23, self.d24, self.d34) = compute_distances_4pts(self.X, self.d)
           
    def __repr__(self):
        write =  "A metric on 4 points, described by the split system: \n"
        write +=  str(self.S) + "\n"
        return write
                 

def compute_distances_4pts(X, d):
    """Compute distances needed for Lipschitz and fundamental polytope calculations
    Called in initialiser of the class to avoid computing twice."""
    points = sorted(X.set)
    d12 = d(points[0], points[1])
    d13 = d(points[0], points[2])
    d14 = d(points[0], points[3])
    d23 = d(points[1], points[2])
    d24 = d(points[1], points[3])   
    d34 = d(points[2], points[3])
    return (d12, d13, d14, d23, d24, d34)
    

def lipschitz_polytope_4pointsset(metric):
    """Constructs the Lipschitz polytope for 4 points sets metrics.
    Provide a MetricOn4Points instance."""
    equation = [[0,1,1,1,1]]
    inequations = ([metric.d12,1,-1,0,0], [metric.d12,-1,1,0,0], [metric.d13,1,0,-1,0], [metric.d13,-1,0,1,0], 
                   [metric.d14,1,0,0,-1], [metric.d14,-1,0,0,1], [metric.d23,0,1,-1,0], [metric.d23,0,-1,1,0],
                   [metric.d24,0,1,0,-1], [metric.d24,0,-1,0,1], [metric.d34,0,0,1,-1], [metric.d34,0,0,-1,1])
    return Polyhedron(eqns=equation, ieqs=inequations)
    
def fundamental_polytope_4pointsset(metric):
    """Constructs the Fundamental polytope for 4 points sets metrics.
    Provide a MetricOn4Points instance."""
    I = metric.X.Ivectors
    points = sorted(metric.X.set)
    e12 = (I[points[0]] - I[points[1]]) / metric.d12
    e13 = (I[points[0]] - I[points[2]]) / metric.d13
    e14 = (I[points[0]] - I[points[3]]) / metric.d14
    e23 = (I[points[1]] - I[points[2]]) / metric.d23
    e24 = (I[points[1]] - I[points[3]]) / metric.d24
    e34 = (I[points[2]] - I[points[3]]) / metric.d34
    e21 = -e12
    e31 = -e13
    e41 = -e14
    e32 = -e23
    e42 = -e24
    e43 = -e34
    return Polyhedron(vertices=[e12, e13, e14, e23, e24, e34, e21, e31, e41, e32, e42, e43])


# List of all possibilities with 0 or 1 weight, up to symmetry  
possibilities = [[1,0,0,0,0,0], [0,0,0,0,1,0], # 1 split                            
                 [1,1,0,0,0,0], [1,0,0,0,1,0], [0,0,0,0,1,1], # 2 splits
                 [1,1,0,0,1,0], [1,1,1,0,0,0], [1,1,0,0,0,1], [1,0,0,0,1,1], # 3 splits
                 [1,1,1,0,1,0], [1,1,0,0,1,1], [1,1,1,1,0,0], [1,0,0,1,1,1], # 4 splits
                 [1,1,1,1,1,0], [1,1,1,0,1,1], # 5 splits
                 [1,1,1,1,1,1]] # 6 splits

### Custom split

In [4]:
## User defined custom split system
# Choose weights in correct order
weights = [1,3,0,2,4,1]

#Compute
M = MetricOn4Points(weights)
rows = [results(M)]
filename = "4pts_weights" + ''.join(map(str, weights)) + ".txt"

# Write output in file
f = open(filename, 'w')
T = table(rows, header_row = header, frame = True)
f.write(str(T.transpose()))
f.close()

In [6]:
#TODO_____________________________________________
# Rewrite/remind splits into file at the beginning and ask user to specify the weights
promt

a = list(map(int,raw_input("\nPlease enter the 6 desired weights separated by spaces : ").strip().split()))[:6] 

### All possibilities

In [5]:
## ALL POSSIBILITIES FOR 0 AND 1 WEIGHTS

# FOR RESULTS DISPLAY
rows = []
# FOR LATEX TABLES spread on 2 pages
rowsleft = []
rowsright = []
cut = 6

## Calculate and store results for the specific split systems we want
for p in possibilities:
    r = results(MetricOn4Points(p))
    rows.append(r)
    rowsleft.append(r[:cut])  
    rowsright.append(r[cut:])
        
## Construct tables and files
Table = table(rows, header_row = header, frame = True, align = 'right')

# Results in txt
description = "Possibilities for splits with weights 0 or 1, eliminating 'symmetrical' cases\n"
f = open("4points_possibilities.txt", 'w')
f.write(description)
f.write(str(Table))
f.close()

# Results in latex
flatex = open("4points_possibilities_latex.tex", 'w')
flatex.write(str(latex(Table)))
flatex.close()

# Results in latex for 2 pages
TableLeft = table(rowsleft, header_row = header[:cut], frame = True, align = 'right')
TableRight = table(rowsright, header_row = header[cut:], frame = True, align = 'right')
flr = open("4points_possibilities_latex_leftright.tex", 'w')
flr.write(str(latex(TableLeft)) + "\n \n")
flr.write(str(latex(TableRight)))
flr.close()

### Test for 2 points metric

In [4]:
def lipschitz_polytope_2pointsset(X, S):
    points = sorted(X.set)
    d12 = S.d_alpha(points[0], points[1])
    return Polyhedron(eqns=[[0,1,1]], ieqs=([d12,1,-1], [d12,-1,1]))

X = FiniteSet(Set(['1', '2']))
S = SplitSystem(X)
S.splits.extend([
Split(X, Set(['1']), 1)])

Lip = lipschitz_polytope_2pointsset(X,S)
fvector_Lip = Lip.f_vector()
intpts_Lip = len(Lip.integral_points())


### Zonotope from Minkowski sum
# and its f-vector, and number of integer points
Z = S.zonotope()
gen_vec = S.zonotope_generating_vectors()
fvector_Z = Z.f_vector()
intpts_Z = len(Z.integral_points())
# Scale zonotope to make it rational and calculate Ehrhart polynomial and number of integer points
Zscaled = X.n * Z
gen_vec_scaled = []
for vec in gen_vec:
    gen_vec_scaled.append(X.n * vec)
h(t) = ehrhart_poly(gen_vec_scaled, X)
intptsscaled = h(1)

 # devrait pas etre egal pour certains cas?
(Lip == Z, Lip, fvector_Lip, intpts_Lip, Z, fvector_Z, intpts_Z, Zscaled, h(t), intptsscaled, Zscaled.f_vector())

(True,
 A 1-dimensional polyhedron in QQ^2 defined as the convex hull of 2 vertices,
 (1, 2, 1),
 1,
 A 1-dimensional polyhedron in QQ^2 defined as the convex hull of 2 vertices,
 (1, 2, 1),
 1,
 A 1-dimensional polyhedron in QQ^2 defined as the convex hull of 2 vertices,
 2*t + 1,
 3,
 (1, 2, 1))

### Test for 3 points metric

In [12]:
def lipschitz_polytope_3pointsset(X, S):
    points = sorted(X.set)
    d12 = S.d_alpha(points[0], points[1])
    d13 = S.d_alpha(points[0], points[2])
    d23 = S.d_alpha(points[1], points[2])
    return Polyhedron(eqns=[[0,1,1,1]], ieqs=([d12,1,-1,0], [d12,-1,1,0], [d13,1,0,-1], [d13,-1,0,1], 
                                             [d23,0,1,-1], [d23,0,-1,1]))
    

X = FiniteSet(Set(['1', '2', '3']))
S = SplitSystem(X)
S.splits.extend([
Split(X, Set(['1']), 1),
#Split(X, Set(['2']), 1),
Split(X, Set(['3']), 1),
#Split(X, Set(['1', '2']), 1),
#Split(X, Set(['1', '3']), 1)
])

Lip = lipschitz_polytope_3pointsset(X,S)
fvector_Lip = Lip.f_vector()
intpts_Lip = len(Lip.integral_points())


### Zonotope from Minkowski sum
# and its f-vector, and number of integer points
Z = S.zonotope()
gen_vec = S.zonotope_generating_vectors()
fvector_Z = Z.f_vector()
intpts_Z = len(Z.integral_points())
# Scale zonotope to make it rational and calculate Ehrhart polynomial and number of integer points
Zscaled = X.n * Z
gen_vec_scaled = []
for vec in gen_vec:
    gen_vec_scaled.append(X.n * vec)
h(t) = ehrhart_poly(gen_vec_scaled, X)
intptsscaled = h(1)

 # devrait pas etre egal pour certains cas?
(Lip == Z, Lip, fvector_Lip, intpts_Lip, Z, fvector_Z, intpts_Z, Zscaled, h(t), intptsscaled, Zscaled.f_vector())

A 2-dimensional polyhedron in QQ^3 defined as the convex hull of 4 vertices (use the .plot() method to plot)

In [65]:
promt

a = list(map(int,raw_input("\nPlease enter the 6 desired weights separated by spaces : ").strip().split()))[:6] 

KeyboardInterrupt: 

In [ ]:
def V_4pointsset(metric):
    """Constructs V(X, d_alpha) for 4 points sets metrics"""
    equations = [[0,1,1,1,1]]
    if metric.d12==0:
        equations.append([0,1,-1,0,0])
    if metric.d13==0:
        equations.append([0,1,0,-1,0])
    if metric.d14==0:
        equations.append([0,1,0,0,-1])
    if metric.d23==0:
        equations.append([0,0,1,-1,0])
    if metric.d24==0:
        equations.append([0,0,1,0,-1])
    if metric.d34==0:
        equations.append([0,0,0,1,-1])
    return Polyhedron(eqns=equations)    